# Predicting propensity_target in Customers_train_enriched

### Notebook automatically generated from your model

Model Gradient Boosted Trees, trained on 2019-03-04 18:54:57.

#### Generated on 2019-03-04 17:58:10.155162

prediction
This notebook will reproduce the steps for a BINARY_CLASSIFICATION on  Customers_train_enriched.
The main objective is to predict the variable propensity_target

#### Warning

The goal of this notebook is to provide an easily readable and explainable code that reproduces the main steps
of training the model. It is not complete: some of the preprocessing done by the DSS visual machine learning is not
replicated in this notebook. This notebook will not give the same results and model performance as the DSS visual machine
learning model.

Let's start with importing the required libs :

In [0]:
import sys
import dataiku
import numpy as np
import pandas as pd
import sklearn as sk
import dataiku.core.pandasutils as pdu
from dataiku.doctor.preprocessing import PCA
from collections import defaultdict, Counter

And tune pandas display options:

In [0]:
pd.set_option('display.width', 3000)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

#### Importing base data

The first step is to get our machine learning dataset:

In [0]:
# We apply the preparation that you defined. You should not modify this.
preparation_steps = []
preparation_output_schema = {u'userModified': False, u'columns': [{u'type': u'bigint', u'name': u'CustomerID'}, {u'type': u'string', u'name': u'CustomerName'}, {u'type': u'date', u'name': u'CustomerBirth'}, {u'type': u'bigint', u'name': u'Age'}, {u'type': u'string', u'name': u'CustomerTitle'}, {u'type': u'string', u'name': u'MaritalStatus'}, {u'type': u'bigint', u'name': u'CustomerIncome'}, {u'type': u'string', u'name': u'CustomerCity'}, {u'type': u'string', u'name': u'CustomerCountry'}, {u'type': u'string', u'name': u'CustomerRegion'}, {u'type': u'string', u'name': u'CustomerSubRegion'}, {u'type': u'string', u'name': u'CustomerEmail'}, {u'type': u'string', u'name': u'CustomerEmail_domain'}, {u'type': u'string', u'name': u'Customer_geopoint'}, {u'type': u'bigint', u'name': u'propensity_target'}, {u'type': u'double', u'name': u'Book_Recency_first'}, {u'type': u'double', u'name': u'Book_Amount_sum'}, {u'type': u'double', u'name': u'Book_Frequency_sum'}, {u'type': u'double', u'name': u'Electronics_Recency_first'}, {u'type': u'double', u'name': u'Electronics_Amount_sum'}, {u'type': u'double', u'name': u'Electronics_Frequency_sum'}, {u'type': u'double', u'name': u'Games_Recency_first'}, {u'type': u'double', u'name': u'Games_Amount_sum'}, {u'type': u'double', u'name': u'Games_Frequency_sum'}, {u'type': u'double', u'name': u'Garden_Recency_first'}, {u'type': u'double', u'name': u'Garden_Amount_sum'}, {u'type': u'double', u'name': u'Garden_Frequency_sum'}, {u'type': u'double', u'name': u'Home_Recency_first'}, {u'type': u'double', u'name': u'Home_Amount_sum'}, {u'type': u'double', u'name': u'Home_Frequency_sum'}, {u'type': u'double', u'name': u'Industrial_Recency_first'}, {u'type': u'double', u'name': u'Industrial_Amount_sum'}, {u'type': u'double', u'name': u'Industrial_Frequency_sum'}, {u'type': u'double', u'name': u'Jewelery_Recency_first'}, {u'type': u'double', u'name': u'Jewelery_Amount_sum'}, {u'type': u'double', u'name': u'Jewelery_Frequency_sum'}, {u'type': u'double', u'name': u'Kids_Recency_first'}, {u'type': u'double', u'name': u'Kids_Amount_sum'}, {u'type': u'double', u'name': u'Kids_Frequency_sum'}, {u'type': u'double', u'name': u'Toys_Recency_first'}, {u'type': u'double', u'name': u'Toys_Amount_sum'}, {u'type': u'double', u'name': u'Toys_Frequency_sum'}, {u'type': u'double', u'name': u'Recency_tot'}, {u'type': u'double', u'name': u'Frequency_tot'}, {u'type': u'double', u'name': u'Amount_tot'}]}

ml_dataset_handle = dataiku.Dataset('Customers_train_enriched')
ml_dataset_handle.set_preparation_steps(preparation_steps, preparation_output_schema)
%time ml_dataset = ml_dataset_handle.get_dataframe(limit = 100000)

print ('Base data has %i rows and %i columns' % (ml_dataset.shape[0], ml_dataset.shape[1]))
# Five first records",
ml_dataset.head(5)

#### Initial data management

The preprocessing aims at making the dataset compatible with modeling.
At the end of this step, we will have a matrix of float numbers, with no missing values.
We'll use the features and the preprocessing steps defined in Models.

Let's only keep selected features

In [0]:
ml_dataset = ml_dataset[[u'Kids_Frequency_sum', u'Amount_tot', u'Jewelery_Amount_sum', u'Toys_Amount_sum', u'Book_Amount_sum', u'Electronics_Frequency_sum', u'MaritalStatus', u'Games_Frequency_sum', u'Electronics_Recency_first', u'Toys_Recency_first', u'Book_Frequency_sum', u'Games_Amount_sum', u'Garden_Recency_first', u'Book_Recency_first', u'Industrial_Amount_sum', u'propensity_target', u'Age', u'Frequency_tot', u'Jewelery_Frequency_sum', u'Kids_Amount_sum', u'Toys_Frequency_sum', u'CustomerTitle', u'Garden_Frequency_sum', u'Garden_Amount_sum', u'Industrial_Frequency_sum', u'Recency_tot', u'Games_Recency_first', u'Home_Recency_first', u'Kids_Recency_first', u'CustomerIncome', u'Home_Frequency_sum', u'Jewelery_Recency_first', u'Home_Amount_sum', u'Industrial_Recency_first', u'Electronics_Amount_sum']]

Let's first coerce categorical columns into unicode, numerical features into floats.

In [0]:
# astype('unicode') does not work as expected

def coerce_to_unicode(x):
    if sys.version_info < (3, 0):
        if isinstance(x, str):
            return unicode(x,'utf-8')
        else:
            return unicode(x)
    else:
        return str(x)


categorical_features = [u'MaritalStatus', u'CustomerTitle']
numerical_features = [u'Kids_Frequency_sum', u'Amount_tot', u'Jewelery_Amount_sum', u'Toys_Amount_sum', u'Book_Amount_sum', u'Electronics_Frequency_sum', u'Games_Frequency_sum', u'Electronics_Recency_first', u'Toys_Recency_first', u'Book_Frequency_sum', u'Games_Amount_sum', u'Garden_Recency_first', u'Book_Recency_first', u'Industrial_Amount_sum', u'Age', u'Frequency_tot', u'Jewelery_Frequency_sum', u'Kids_Amount_sum', u'Toys_Frequency_sum', u'Garden_Frequency_sum', u'Garden_Amount_sum', u'Industrial_Frequency_sum', u'Recency_tot', u'Games_Recency_first', u'Home_Recency_first', u'Kids_Recency_first', u'CustomerIncome', u'Home_Frequency_sum', u'Jewelery_Recency_first', u'Home_Amount_sum', u'Industrial_Recency_first', u'Electronics_Amount_sum']
text_features = []
from dataiku.doctor.utils import datetime_to_epoch
for feature in categorical_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in text_features:
    ml_dataset[feature] = ml_dataset[feature].apply(coerce_to_unicode)
for feature in numerical_features:
    if ml_dataset[feature].dtype == np.dtype('M8[ns]'):
        ml_dataset[feature] = datetime_to_epoch(ml_dataset[feature])
    else:
        ml_dataset[feature] = ml_dataset[feature].astype('double')

We are now going to handle the target variable and store it in a new variable:

In [0]:
target_map = {u'1': 1, u'0': 0}
ml_dataset['__target__'] = ml_dataset['propensity_target'].map(str).map(target_map)
del ml_dataset['propensity_target']


# Remove rows for which the target is unknown.
ml_dataset = ml_dataset[~ml_dataset['__target__'].isnull()]

#### Cross-validation strategy

The dataset needs to be split into 2 new sets, one that will be used for training the model (train set)
and another that will be used to test its generalization capability (test set)

This is a simple cross-validation strategy.

In [0]:
train, test = pdu.split_train_valid(ml_dataset, prop=0.8)
print ('Train data has %i rows and %i columns' % (train.shape[0], train.shape[1]))
print ('Test data has %i rows and %i columns' % (test.shape[0], test.shape[1]))

#### Features preprocessing

The first thing to do at the features level is to handle the missing values.
Let's reuse the settings defined in the model

In [0]:
drop_rows_when_missing = []
impute_when_missing = [{'impute_with': u'CONSTANT', 'feature': u'Kids_Frequency_sum', 'value': 0.0}, {'impute_with': u'MEAN', 'feature': u'Amount_tot'}, {'impute_with': u'CONSTANT', 'feature': u'Jewelery_Amount_sum', 'value': 0.0}, {'impute_with': u'CONSTANT', 'feature': u'Toys_Amount_sum', 'value': 0.0}, {'impute_with': u'CONSTANT', 'feature': u'Book_Amount_sum', 'value': 0.0}, {'impute_with': u'CONSTANT', 'feature': u'Electronics_Frequency_sum', 'value': 0.0}, {'impute_with': u'CONSTANT', 'feature': u'Games_Frequency_sum', 'value': 0.0}, {'impute_with': u'MEAN', 'feature': u'Electronics_Recency_first'}, {'impute_with': u'MEAN', 'feature': u'Toys_Recency_first'}, {'impute_with': u'CONSTANT', 'feature': u'Book_Frequency_sum', 'value': 0.0}, {'impute_with': u'CONSTANT', 'feature': u'Games_Amount_sum', 'value': 0.0}, {'impute_with': u'MEAN', 'feature': u'Garden_Recency_first'}, {'impute_with': u'MEAN', 'feature': u'Book_Recency_first'}, {'impute_with': u'CONSTANT', 'feature': u'Industrial_Amount_sum', 'value': 0.0}, {'impute_with': u'MEAN', 'feature': u'Age'}, {'impute_with': u'MEAN', 'feature': u'Frequency_tot'}, {'impute_with': u'CONSTANT', 'feature': u'Jewelery_Frequency_sum', 'value': 0.0}, {'impute_with': u'CONSTANT', 'feature': u'Kids_Amount_sum', 'value': 0.0}, {'impute_with': u'CONSTANT', 'feature': u'Toys_Frequency_sum', 'value': 0.0}, {'impute_with': u'CONSTANT', 'feature': u'Garden_Frequency_sum', 'value': 0.0}, {'impute_with': u'CONSTANT', 'feature': u'Garden_Amount_sum', 'value': 0.0}, {'impute_with': u'CONSTANT', 'feature': u'Industrial_Frequency_sum', 'value': 0.0}, {'impute_with': u'MEAN', 'feature': u'Recency_tot'}, {'impute_with': u'MEAN', 'feature': u'Games_Recency_first'}, {'impute_with': u'MEAN', 'feature': u'Home_Recency_first'}, {'impute_with': u'MEAN', 'feature': u'Kids_Recency_first'}, {'impute_with': u'MEAN', 'feature': u'CustomerIncome'}, {'impute_with': u'CONSTANT', 'feature': u'Home_Frequency_sum', 'value': 0.0}, {'impute_with': u'MEAN', 'feature': u'Jewelery_Recency_first'}, {'impute_with': u'CONSTANT', 'feature': u'Home_Amount_sum', 'value': 0.0}, {'impute_with': u'MEAN', 'feature': u'Industrial_Recency_first'}, {'impute_with': u'CONSTANT', 'feature': u'Electronics_Amount_sum', 'value': 0.0}]

# Features for which we drop rows with missing values"
for feature in drop_rows_when_missing:
    train = train[train[feature].notnull()]
    test = test[test[feature].notnull()]
    print ('Dropped missing records in %s' % feature)

# Features for which we impute missing values"
for feature in impute_when_missing:
    if feature['impute_with'] == 'MEAN':
        v = train[feature['feature']].mean()
    elif feature['impute_with'] == 'MEDIAN':
        v = train[feature['feature']].median()
    elif feature['impute_with'] == 'CREATE_CATEGORY':
        v = 'NULL_CATEGORY'
    elif feature['impute_with'] == 'MODE':
        v = train[feature['feature']].value_counts().index[0]
    elif feature['impute_with'] == 'CONSTANT':
        v = feature['value']
    train[feature['feature']] = train[feature['feature']].fillna(v)
    test[feature['feature']] = test[feature['feature']].fillna(v)
    print ('Imputed missing values in feature %s with value %s' % (feature['feature'], coerce_to_unicode(v)))

We can now handle the categorical features (still using the settings defined in Models):

Let's dummy-encode the following features.
A binary column is created for each of the 100 most frequent values.

In [0]:
LIMIT_DUMMIES = 100

categorical_to_dummy_encode = [u'MaritalStatus', u'CustomerTitle']

# Only keep the top 100 values
def select_dummy_values(train, features):
    dummy_values = {}
    for feature in categorical_to_dummy_encode:
        values = [
            value
            for (value, _) in Counter(train[feature]).most_common(LIMIT_DUMMIES)
        ]
        dummy_values[feature] = values
    return dummy_values

DUMMY_VALUES = select_dummy_values(train, categorical_to_dummy_encode)

def dummy_encode_dataframe(df):
    for (feature, dummy_values) in DUMMY_VALUES.items():
        for dummy_value in dummy_values:
            dummy_name = u'%s_value_%s' % (feature, coerce_to_unicode(dummy_value))
            df[dummy_name] = (df[feature] == dummy_value).astype(float)
        del df[feature]
        print ('Dummy-encoded feature %s' % feature)

dummy_encode_dataframe(train)

dummy_encode_dataframe(test)

Let's rescale numerical features

In [0]:
rescale_features = {u'Kids_Frequency_sum': u'AVGSTD', u'Toys_Frequency_sum': u'AVGSTD', u'Jewelery_Amount_sum': u'AVGSTD', u'Book_Amount_sum': u'AVGSTD', u'Games_Recency_first': u'AVGSTD', u'Games_Frequency_sum': u'AVGSTD', u'Kids_Amount_sum': u'AVGSTD', u'Book_Frequency_sum': u'AVGSTD', u'Games_Amount_sum': u'AVGSTD', u'Garden_Recency_first': u'AVGSTD', u'Book_Recency_first': u'AVGSTD', u'Industrial_Amount_sum': u'AVGSTD', u'Toys_Recency_first': u'AVGSTD', u'Age': u'AVGSTD', u'CustomerIncome': u'AVGSTD', u'Jewelery_Frequency_sum': u'AVGSTD', u'Amount_tot': u'AVGSTD', u'Electronics_Recency_first': u'AVGSTD', u'Garden_Frequency_sum': u'AVGSTD', u'Garden_Amount_sum': u'AVGSTD', u'Industrial_Frequency_sum': u'AVGSTD', u'Recency_tot': u'AVGSTD', u'Toys_Amount_sum': u'AVGSTD', u'Electronics_Frequency_sum': u'AVGSTD', u'Home_Recency_first': u'AVGSTD', u'Kids_Recency_first': u'AVGSTD', u'Frequency_tot': u'AVGSTD', u'Home_Frequency_sum': u'AVGSTD', u'Jewelery_Recency_first': u'AVGSTD', u'Home_Amount_sum': u'AVGSTD', u'Industrial_Recency_first': u'AVGSTD', u'Electronics_Amount_sum': u'AVGSTD'}
for (feature_name, rescale_method) in rescale_features.items():
    if rescale_method == 'MINMAX':
        _min = train[feature_name].min()
        _max = train[feature_name].max()
        scale = _max - _min
        shift = _min
    else:
        shift = train[feature_name].mean()
        scale = train[feature_name].std()
    if scale == 0.:
        del train[feature_name]
        del test[feature_name]
        print ('Feature %s was dropped because it has no variance' % feature_name)
    else:
        print ('Rescaled %s' % feature_name)
        train[feature_name] = (train[feature_name] - shift).astype(np.float64) / scale
        test[feature_name] = (test[feature_name] - shift).astype(np.float64) / scale

#### Modeling

Before actually creating our model, we need to split the datasets into their features and labels parts:

In [0]:
train_X = train.drop('__target__', axis=1)
test_X = test.drop('__target__', axis=1)

train_Y = np.array(train['__target__'])
test_Y = np.array(test['__target__'])

Now we can finally create our model !

In [0]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(
                    random_state = 1337,
                    verbose = 0,
                    n_estimators = 100,
                    learning_rate = 0.1,
                    loss = 'deviance',
                    max_depth = 3
                   )

... And train it

In [0]:
%time clf.fit(train_X, train_Y)

Build up our result dataset

The model is now trained, we can apply it to our test set:

In [0]:
%time _predictions = clf.predict(test_X)
%time _probas = clf.predict_proba(test_X)
predictions = pd.Series(data=_predictions, index=test_X.index, name='predicted_value')
cols = [
    u'probability_of_value_%s' % label
    for (_, label) in sorted([(int(target_map[label]), label) for label in target_map])
]
probabilities = pd.DataFrame(data=_probas, index=test_X.index, columns=cols)

# Build scored dataset
results_test = test_X.join(predictions, how='left')
results_test = results_test.join(probabilities, how='left')
results_test = results_test.join(test['__target__'], how='left')
results_test = results_test.rename(columns= {'__target__': 'propensity_target'})

#### Results

You can measure the model's accuracy:

In [0]:
from dataiku.doctor.utils.metrics import mroc_auc_score
test_Y_ser = pd.Series(test_Y)
print ('AUC value:', mroc_auc_score(test_Y_ser, _probas))

We can also view the predictions directly.
Since scikit-learn only predicts numericals, the labels have been mapped to 0,1,2 ...
We need to 'reverse' the mapping to display the initial labels.

In [0]:
inv_map = { target_map[label] : label for label in target_map}
predictions.map(inv_map)

That's it. It's now up to you to tune your preprocessing, your algo, and your analysis !
